# VacationPy
* In this code, google maps is used to show the humidity at the random cities that were collected in WeatherPy. The weater data that was collected is used to locate several cities that meet an ideal vacation climate. The ideal weather chosen is between 70-83F with <10% cloudiness, wind <10 mph, and humidity < 45%. For each ideal climate location, the nearest hotel is found and marked on a map.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

#Using some tips from StackOverflow to suppress warnings
#https://stackoverflow.com/questions/14463277/how-to-disable-python-warnings
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import g_key

### Store Part I results into DataFrame

In [2]:
cities_list = pd.read_csv("../WeatherPY/out_put/cities.csv")
cities_list.head()

,City,Country,Latitude,Longitude,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed(mph),City Date/Time
0,Gushikawa,JP,26.35,127.87,87.80,74,20,13.87,1596413633
1,Opuwo,NaN,-18.06,13.84,58.37,34,0,7.83,1596413371
2,Hermanus,ZA,-34.42,19.23,48.99,97,0,1.01,1596413633
3,Wattegama,LK,6.80,81.48,78.75,68,100,11.50,1596413633
4,Hithadhoo,MV,-0.60,73.08,83.10,75,51,10.18,1596413342


### Humidity Heatmap

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key) 

#Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
humidity_map = gmaps.figure(layout=figure_layout)

#Creating a list of city coordinates from the cities list
locations = cities_list[["Latitude", "Longitude"]].astype(float)
#Geting the humidity 
humidity = cities_list["Humidity(%)"].astype(float)

#Using city locations and humidity to creat a heat map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
humidity_map.add_layer(heat_layer)

# Displaying heatmap
humidity_map 

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria

In [5]:
#My ideal weather is between 70-83F with <10% cloudiness, wind <10 mph, and humidity < 45%.
#Find all the cites in the list with my ideal weather. 
ideal_weather = cities_list.loc[(cities_list["Cloudiness(%)"] < 15) & ((cities_list["Max Temp(F)"] > 70) & (cities_list["Max Temp(F)"] < 83)) & (cities_list["Wind Speed(mph)"] < 10) & (cities_list["Humidity(%)"] < 45)]
ideal_weather 

,City,Country,Latitude,Longitude,Max Temp(F),Humidity(%),Cloudiness(%),Wind Speed(mph),City Date/Time
28,Jacareacanga,BR,-6.22,-57.75,76.33,43,4,0.89,1596413640
78,Coxim,BR,-18.51,-54.76,77.34,33,0,7.25,1596413654
396,Torbat-e Jām,IR,35.24,60.62,71.92,22,0,9.24,1596413735
403,Mizdah,LY,31.45,12.98,76.48,25,2,5.86,1596413737
417,Pozo Colorado,PY,-23.49,-58.80,76.78,43,1,7.09,1596413741
474,Dongsheng,CN,39.82,109.98,71.11,35,0,0.18,1596413755
514,Gazojak,TM,41.19,61.40,80.33,23,0,7.72,1596413766


### Hotel Map

In [6]:
#Creating a hotel dataframe and adding a hotel name
hotel_df = ideal_weather
hotel_df["Hotel Name"] =""

#Using google maps API to find a hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Find a hotel near the cities with ideal weather     
for index, row in hotel_df.iterrows():
    
    #Setting the parameters for the hotel search URL
    locations = f"{row['Latitude']}, {row['Longitude']}"
    params = {"location": locations,"types": "lodging","radius": 5000,"key": g_key}
    
    #Assemble the URL and make API request
    hotel_results = requests.get(base_url, params=params)
    hotel = hotel_results.json()
    
    #List the city and the closest hotel
    print(f"Finding a hotel near: {row['City']}.")
    try:
        print(f"Closest lodging is {hotel['results'][0]['name']}.")   
        hotel_df.loc[index, 'Hotel Name'] = hotel['results'][0]['name']
    except:
        print("No lodging close by... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = "N/A"
    print("------------")

Finding a hotel near: Jacareacanga.
Closest lodging is Casa do Ian😎.
------------
Finding a hotel near: Coxim.
Closest lodging is Hotel Avenida.
------------
Finding a hotel near: Torbat-e Jām.
Closest lodging is ساختمان مسکونی فلورا.
------------
Finding a hotel near: Mizdah.
Closest lodging is محمدالمبروك الاريقط.
------------
Finding a hotel near: Pozo Colorado.
Closest lodging is Hospedaje la Amistad.
------------
Finding a hotel near: Dongsheng.
Closest lodging is Crowne Plaza Ordos.
------------
Finding a hotel near: Gazojak.
No lodging close by... skipping.
------------


In [7]:
#Take out the cites that don't have lodging nearby
no_hotel = hotel_df[hotel_df["Hotel Name"] == "N/A"].index
new_hotel_list = hotel_df.drop(no_hotel)
final_hotel_list = new_hotel_list.dropna()
final_hotel_list = final_hotel_list[["City", "Country", "Latitude", "Longitude", "Hotel Name"]]
final_hotel_list

,City,Country,Latitude,Longitude,Hotel Name
28,Jacareacanga,BR,-6.22,-57.75,Casa do Ian😎
78,Coxim,BR,-18.51,-54.76,Hotel Avenida
396,Torbat-e Jām,IR,35.24,60.62,ساختمان مسکونی فلورا
403,Mizdah,LY,31.45,12.98,محمدالمبروك الاريقط
417,Pozo Colorado,PY,-23.49,-58.80,Hospedaje la Amistad
474,Dongsheng,CN,39.82,109.98,Crowne Plaza Ordos


In [8]:
# Using the template add the hotel info to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in final_hotel_list.iterrows()]

#Get the Hotel locations 
locations =final_hotel_list[["Latitude", "Longitude"]]

In [9]:
# Assign the marker and info box layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the humidity map
humidity_map.add_layer(markers)

#Show the map with the hotels in the ideal weather locations
humidity_map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…